In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-2.0.1-py3-none-any.whl size=193267 sha256=4416e2a30d0bb51f046bc39f5b46e3c4ca32915ffdb1432ba33a0c3d9814c383
  Stored in directory: /root/.cache/pip/wheels/94/75/88/b853cf33b0053b0a001dca55b74d515048b7656e736364eb57
Successfully built pyod


In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn import metrics
import scipy.io
import warnings
from pyod.models.knn import KNN
from pyod.models.kde import KDE
from pyod.models.lof import LOF
from pyod.models.hbos import HBOS
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.kpca import KPCA
from pyod.models.cblof import CBLOF
from pyod.models.iforest import IForest
from pyod.models.loda import LODA
from pyod.models.lunar import LUNAR

# Suppress all warnings
warnings.filterwarnings("ignore")

# Define function to convert Torch tensor to NumPy array
def torch_to_numpy(tensor):
    return tensor.detach().cpu().numpy()

# Define function to evaluate models
def evaluate_models(algorithms, datasets, device):
    results = {}

    for dataset_name, dataset_path in datasets.items():
        print(f"Evaluating on dataset: {dataset_name}")

        # Load dataset
        df = scipy.io.loadmat(dataset_path)
        numeric = torch.FloatTensor(df['X'])
        labels = (df['y']).astype(float).reshape(-1)

        # Convert Torch tensors to NumPy arrays
        numeric_np = torch_to_numpy(numeric)
        labels_np = labels.astype(float)

        # Split dataset for training
        N = 2 * len(labels_np) // 3  # Use 2/3 of data for training

        # Create data loader
        data_loader = DataLoader(numeric, batch_size=1)

        # Initialize results for current dataset
        dataset_results = {
            'dataset': dataset_name,
            'algorithms': {}
        }

        # Evaluate each algorithm
        for alg_name, clf in algorithms.items():
            print(f"Running {alg_name}...")

            # Fit the model (convert input to NumPy arrays)
            clf.fit(numeric_np[labels_np == 0])

            # Get predictions and scores
            y_test_pred = []
            y_test_scores = []
            err0 = 0
            err1 = 0
            i = 0

            for data in data_loader:
                # Get the prediction on the test data
                y_pred = clf.predict(torch_to_numpy(data.to(device)))
                y_test_pred.append(y_pred)  # Outlier labels (0 or 1)

                # Get outlier scores
                y_scores = torch.tensor(clf.decision_function(torch_to_numpy(data.to(device)))[0]).type(torch.float)
                y_test_scores.append(y_scores)

                # Count false positives and false negatives
                if y_pred == 1 and labels[i] == 0:
                    err0 += 1
                elif y_pred == 0 and labels[i] == 1:
                    err1 += 1

                i += 1

            # Calculate evaluation metrics
            scores = np.array([s.cpu() for s in y_test_scores])
            auc = metrics.roc_auc_score(labels, scores)
            pr = metrics.average_precision_score(labels, scores)

            # Store results for current algorithm
            dataset_results['algorithms'][alg_name] = {
                'ROC-AUC': auc,
                'AUC-PR': pr,
                'False Positives': err0,
                'False Negatives': err1
            }

            # Print results for current algorithm
            print(f"{alg_name}:")
            print(f"ROC-AUC: {auc} | AUC-PR: {pr}")
            print(f"False Positives: {err0} | False Negatives: {err1}")
            print()

        # Store results for current dataset
        results[dataset_name] = dataset_results

    return results

# Define algorithms to evaluate
algorithms = {
    'KDE': KDE(),
    'PCA': PCA(),
    'LOF': LOF(),
    'CBLOF': CBLOF(),
    'HBOS': HBOS(),
    'kNN': KNN(),
    'IForest': IForest(),
    'LODA': LODA(),
    'LUNAR': LUNAR()
}

# Define datasets
datasets = {
    'ionosphere': '/content/drive/MyDrive/FTP/data/ionosphere.mat',
    'cardio': '/content/drive/MyDrive/FTP/data/cardio.mat',
    'statlog': '/content/drive/MyDrive/FTP/data/statlog.mat',
    'satimage-2': '/content/drive/MyDrive/FTP/data/satimage-2.mat',
    'mammography': '/content/drive/MyDrive/FTP/data/mammography.mat',
    'pima': '/content/drive/MyDrive/FTP/data/pima.mat'
}

# Define device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Run evaluation
results = evaluate_models(algorithms, datasets, device)

# Print or process results as needed
print("Evaluation results:")
print(results)

Evaluating on dataset: ionosphere
Running KDE...
KDE:
ROC-AUC: 0.9538271604938271 | AUC-PR: 0.9415117296252272
False Positives: 23 | False Negatives: 19

Running PCA...
PCA:
ROC-AUC: 0.8982363315696649 | AUC-PR: 0.8623693895861112
False Positives: 23 | False Negatives: 40

Running LOF...
LOF:
ROC-AUC: 0.9573544973544974 | AUC-PR: 0.9283310389409921
False Positives: 19 | False Negatives: 17

Running CBLOF...
CBLOF:
ROC-AUC: 0.9829629629629629 | AUC-PR: 0.9746971291038911
False Positives: 23 | False Negatives: 5

Running HBOS...
HBOS:
ROC-AUC: 0.7707231040564374 | AUC-PR: 0.6105088618606221
False Positives: 23 | False Negatives: 90

Running kNN...
kNN:
ROC-AUC: 0.9781657848324515 | AUC-PR: 0.9678781535602966
False Positives: 18 | False Negatives: 12

Running IForest...
IForest:
ROC-AUC: 0.9246913580246914 | AUC-PR: 0.8750585218062276
False Positives: 23 | False Negatives: 31

Running LODA...
LODA:
ROC-AUC: 0.9058906525573193 | AUC-PR: 0.8597099187613483
False Positives: 23 | False Negati

In [ ]:
# Evaluating on dataset: ionosphere
# Running KDE...
# KDE:
# ROC-AUC: 0.9538271604938271 | AUC-PR: 0.9415117296252272
# False Positives: 23 | False Negatives: 19

# Running PCA...
# PCA:
# ROC-AUC: 0.8982363315696649 | AUC-PR: 0.8623693895861112
# False Positives: 23 | False Negatives: 40

# Running LOF...
# LOF:
# ROC-AUC: 0.9573544973544974 | AUC-PR: 0.9283310389409921
# False Positives: 19 | False Negatives: 17

# Running CBLOF...
# CBLOF:
# ROC-AUC: 0.9829629629629629 | AUC-PR: 0.9746971291038911
# False Positives: 23 | False Negatives: 5

# Running HBOS...
# HBOS:
# ROC-AUC: 0.7707231040564374 | AUC-PR: 0.6105088618606221
# False Positives: 23 | False Negatives: 90

# Running kNN...
# kNN:
# ROC-AUC: 0.9781657848324515 | AUC-PR: 0.9678781535602966
# False Positives: 18 | False Negatives: 12

# Running IForest...
# IForest:
# ROC-AUC: 0.9246913580246914 | AUC-PR: 0.8750585218062276
# False Positives: 23 | False Negatives: 31

# Running LODA...
# LODA:
# ROC-AUC: 0.9058906525573193 | AUC-PR: 0.8597099187613483
# False Positives: 23 | False Negatives: 33

# Running LUNAR...
# LUNAR:
# ROC-AUC: 0.9628218694885362 | AUC-PR: 0.9529540973429316
# False Positives: 23 | False Negatives: 13

# Evaluating on dataset: cardio
# Running KDE...
# KDE:
# ROC-AUC: 0.9855671518813511 | AUC-PR: 0.9561216776292831
# False Positives: 166 | False Negatives: 9

# Running PCA...
# PCA:
# ROC-AUC: 0.9660155177149136 | AUC-PR: 0.7577906002476548
# False Positives: 166 | False Negatives: 19

# Running LOF...
# LOF:
# ROC-AUC: 0.9466561384235099 | AUC-PR: 0.6509003305293414
# False Positives: 143 | False Negatives: 31

# Running CBLOF...
# CBLOF:
# ROC-AUC: 0.9476654765174402 | AUC-PR: 0.6298219735938506
# False Positives: 166 | False Negatives: 23

# Running HBOS...
# HBOS:
# ROC-AUC: 0.8751407580335072 | AUC-PR: 0.5443785572805827
# False Positives: 166 | False Negatives: 71

# Running kNN...
# kNN:
# ROC-AUC: 0.9388320516341664 | AUC-PR: 0.6908405065485204
# False Positives: 133 | False Negatives: 45

# Running IForest...
# IForest:
# ROC-AUC: 0.9622390826695962 | AUC-PR: 0.7252368515081321
# False Positives: 166 | False Negatives: 16

# Running LODA...
# LODA:
# ROC-AUC: 0.9118580060422962 | AUC-PR: 0.586148505133691
# False Positives: 166 | False Negatives: 59

# Running LUNAR...
# LUNAR:
# ROC-AUC: 0.9928213402911288 | AUC-PR: 0.9327725944559088
# False Positives: 166 | False Negatives: 1

# Evaluating on dataset: statlog
# Running KDE...
# KDE:
# ROC-AUC: 1.0 | AUC-PR: 1.0
# False Positives: 0 | False Negatives: 0

# Running PCA...
# PCA:
# ROC-AUC: 0.6623482475701077 | AUC-PR: 0.6867465616586752
# False Positives: 440 | False Negatives: 952

# Running LOF...
# LOF:
# ROC-AUC: 0.8611273503399369 | AUC-PR: 0.8373684227332266
# False Positives: 383 | False Negatives: 613

# Running CBLOF...
# CBLOF:
# ROC-AUC: 0.8820523596405864 | AUC-PR: 0.8179614574436158
# False Positives: 440 | False Negatives: 661

# Running HBOS...
# HBOS:
# ROC-AUC: 0.8690012487210212 | AUC-PR: 0.8139035674646938
# False Positives: 440 | False Negatives: 688

# Running kNN...
# kNN:
# ROC-AUC: 0.8876000350142088 | AUC-PR: 0.8473344834005452
# False Positives: 396 | False Negatives: 589

# Running IForest...
# IForest:
# ROC-AUC: 0.8113028903246898 | AUC-PR: 0.7778901045254049
# False Positives: 440 | False Negatives: 760

# Running LODA...
# LODA:
# ROC-AUC: 0.7032009306455165 | AUC-PR: 0.7281285247404473
# False Positives: 440 | False Negatives: 831

# Running LUNAR...
# LUNAR:
# ROC-AUC: 0.828802625708379 | AUC-PR: 0.7906591713660055
# False Positives: 440 | False Negatives: 689

# Evaluating on dataset: satimage-2
# Running KDE...
# KDE:
# ROC-AUC: 1.0 | AUC-PR: 0.9999999999999999
# False Positives: 0 | False Negatives: 0

# Running PCA...
# PCA:
# ROC-AUC: 0.9784235770519838 | AUC-PR: 0.882149581593636
# False Positives: 574 | False Negatives: 5

# Running LOF...
# LOF:
# ROC-AUC: 0.9944541639228252 | AUC-PR: 0.9255263259060145
# False Positives: 495 | False Negatives: 2

# Running CBLOF...
# CBLOF:
# ROC-AUC: 0.9980514629999115 | AUC-PR: 0.9657384926464789
# False Positives: 574 | False Negatives: 0

# Running HBOS...
# HBOS:
# ROC-AUC: 0.9713149799003371 | AUC-PR: 0.7343087407345981
# False Positives: 574 | False Negatives: 5

# Running kNN...
# kNN:
# ROC-AUC: 0.9992739058215308 | AUC-PR: 0.976150992382446
# False Positives: 498 | False Negatives: 0

# Running IForest...
# IForest:
# ROC-AUC: 0.9924589406642226 | AUC-PR: 0.8998118602197456
# False Positives: 574 | False Negatives: 2

# Running LODA...
# LODA:
# ROC-AUC: 0.9864069272578949 | AUC-PR: 0.8975595638711223
# False Positives: 574 | False Negatives: 2

# Running LUNAR...
# LUNAR:
# ROC-AUC: 0.9963879578939091 | AUC-PR: 0.9543415438300795
# False Positives: 574 | False Negatives: 1

# Evaluating on dataset: mammography
# Running KDE...
# KDE:
# ROC-AUC: 0.871572158958866 | AUC-PR: 0.3281732319834067
# False Positives: 1093 | False Negatives: 106

# Running PCA...
# PCA:
# ROC-AUC: 0.8955950394016859 | AUC-PR: 0.29073174925183026
# False Positives: 1093 | False Negatives: 79

# Running LOF...
# LOF:
# ROC-AUC: 0.8637963999746479 | AUC-PR: 0.24308555483317615
# False Positives: 992 | False Negatives: 103

# Running CBLOF...
# CBLOF:
# ROC-AUC: 0.8356270114578271 | AUC-PR: 0.18301829390052235
# False Positives: 1093 | False Negatives: 118

# Running HBOS...
# HBOS:
# ROC-AUC: 0.8523868125831873 | AUC-PR: 0.1495775149374076
# False Positives: 1083 | False Negatives: 110

# Running kNN...
# kNN:
# ROC-AUC: 0.8809327882590723 | AUC-PR: 0.3097994306882719
# False Positives: 944 | False Negatives: 90

# Running IForest...
# IForest:
# ROC-AUC: 0.8834822780442115 | AUC-PR: 0.2708976859025019
# False Positives: 1093 | False Negatives: 85

# Running LODA...
# LODA:
# ROC-AUC: 0.8802350720779724 | AUC-PR: 0.301493291130438
# False Positives: 1093 | False Negatives: 73

# Running LUNAR...
# LUNAR:
# ROC-AUC: 0.8450758456045465 | AUC-PR: 0.35841674971547355
# False Positives: 1093 | False Negatives: 123

# Evaluating on dataset: pima
# Running KDE...
# KDE:
# ROC-AUC: 1.0 | AUC-PR: 1.0
# False Positives: 0 | False Negatives: 0

# Running PCA...
# PCA:
# ROC-AUC: 0.7162611940298508 | AUC-PR: 0.5443456030084702
# False Positives: 50 | False Negatives: 197

# Running LOF...
# LOF:
# ROC-AUC: 0.6865820895522388 | AUC-PR: 0.5365491116457408
# False Positives: 49 | False Negatives: 184

# Running CBLOF...
# CBLOF:
# ROC-AUC: 0.6622014925373134 | AUC-PR: 0.5182771520957361
# False Positives: 50 | False Negatives: 180

# Running HBOS...
# HBOS:
# ROC-AUC: 0.7030858208955224 | AUC-PR: 0.541541650655659
# False Positives: 50 | False Negatives: 190

# Running kNN...
# kNN:
# ROC-AUC: 0.7125820895522388 | AUC-PR: 0.5677964165705971
# False Positives: 41 | False Negatives: 186

# Running IForest...
# IForest:
# ROC-AUC: 0.743134328358209 | AUC-PR: 0.583270825555484
# False Positives: 50 | False Negatives: 186

# Running LODA...
# LODA:
# ROC-AUC: 0.6546791044776119 | AUC-PR: 0.4486134811617549
# False Positives: 50 | False Negatives: 226

# Running LUNAR...
# LUNAR:
# ROC-AUC: 0.6143731343283583 | AUC-PR: 0.4442779977234344
# False Positives: 50 | False Negatives: 219

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn import metrics
import warnings
from pyod.models.knn import KNN
from pyod.models.kde import KDE
from pyod.models.lof import LOF
from pyod.models.hbos import HBOS
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.kpca import KPCA
from pyod.models.cblof import CBLOF
from pyod.models.iforest import IForest
from pyod.models.loda import LODA
from pyod.models.lunar import LUNAR

# Suppress all warnings
warnings.filterwarnings("ignore")

# Function to load syn dataset
def load_syn_dataset(data_file, label_file):
    with open(data_file, 'r') as f:
        lines = f.readlines()
        numeric = np.array([[float(num) for num in line.strip().split()] for line in lines])

    with open(label_file, 'r') as f:
        labels = np.array([int(label.strip()) for label in f.readlines()])

    return numeric, labels

# Define function to convert Torch tensor to NumPy array
def torch_to_numpy(tensor):
    return tensor.detach().cpu().numpy()

# Define function to evaluate models
def evaluate_models(algorithms, dataset_path, label_path, device):
    # Load dataset
    numeric, labels = load_syn_dataset(dataset_path, label_path)
    numeric = torch.FloatTensor(numeric)
    labels = labels.astype(float)

    # Convert Torch tensors to NumPy arrays
    numeric_np = torch_to_numpy(numeric)
    labels_np = labels.astype(float)

    # Split dataset for training
    N = 2 * len(labels_np) // 3  # Use 2/3 of data for training

    # Create data loader
    data_loader = DataLoader(numeric, batch_size=1)

    results = {}

    # Evaluate each algorithm
    for alg_name, clf in algorithms.items():
        print(f"Running {alg_name}...")

        # Fit the model (convert input to NumPy arrays)
        clf.fit(numeric_np[labels_np == 0])

        # Get predictions and scores
        y_test_pred = []
        y_test_scores = []
        err0 = 0
        err1 = 0
        i = 0

        for data in data_loader:
            # Get the prediction on the test data
            y_pred = clf.predict(torch_to_numpy(data.to(device)))
            y_test_pred.append(y_pred)  # Outlier labels (0 or 1)

            # Get outlier scores
            y_scores = torch.tensor(clf.decision_function(torch_to_numpy(data.to(device)))[0]).type(torch.float)
            y_test_scores.append(y_scores)

            # Count false positives and false negatives
            if y_pred == 1 and labels[i] == 0:
                err0 += 1
            elif y_pred == 0 and labels[i] == 1:
                err1 += 1

            i += 1

        # Calculate evaluation metrics
        scores = np.array([s.cpu() for s in y_test_scores])
        auc = metrics.roc_auc_score(labels, scores)
        pr = metrics.average_precision_score(labels, scores)

        # Store results for current algorithm
        results[alg_name] = {
            'ROC-AUC': auc,
            'AUC-PR': pr,
            'False Positives': err0,
            'False Negatives': err1
        }

        # Print results for current algorithm
        print(f"{alg_name}:")
        print(f"ROC-AUC: {auc} | AUC-PR: {pr}")
        print(f"False Positives: {err0} | False Negatives: {err1}")
        print()

    return results

# Define algorithms to evaluate
algorithms = {
    'KDE': KDE(),
    'PCA': PCA(),
    'LOF': LOF(),
    'CBLOF': CBLOF(),
    'HBOS': HBOS(),
    'kNN': KNN(),
    'IForest': IForest(),
    'LODA': LODA(),
    'LUNAR': LUNAR()
}

# Define paths to syn dataset files
syn_data_file = '/content/drive/MyDrive/FTP/data/syn.txt'
syn_label_file = '/content/drive/MyDrive/FTP/data/synlabel.txt'

# Define device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Run evaluation
results = evaluate_models(algorithms, syn_data_file, syn_label_file, device)

# Print or process results as needed
print("Evaluation results:")
print(results)

Running KDE...
KDE:
ROC-AUC: 0.5524182222222221 | AUC-PR: 0.1944889957384754
False Positives: 900 | False Negatives: 826

Running PCA...
PCA:
ROC-AUC: 0.518875611111111 | AUC-PR: 0.15527543728456505
False Positives: 900 | False Negatives: 866

Running LOF...
LOF:
ROC-AUC: 0.6199740555555555 | AUC-PR: 0.22148765563972567
False Positives: 734 | False Negatives: 798

Running CBLOF...
CBLOF:
ROC-AUC: 0.5491408888888889 | AUC-PR: 0.15972168562991268
False Positives: 900 | False Negatives: 816

Running HBOS...
HBOS:
ROC-AUC: 0.5483418888888889 | AUC-PR: 0.12633453105207998
False Positives: 504 | False Negatives: 867

Running kNN...
kNN:
ROC-AUC: 0.6459407777777777 | AUC-PR: 0.23531530481594098
False Positives: 552 | False Negatives: 818

Running IForest...
IForest:
ROC-AUC: 0.5528506666666667 | AUC-PR: 0.1800694026637641
False Positives: 899 | False Negatives: 826

Running LODA...
LODA:
ROC-AUC: 0.5328418333333333 | AUC-PR: 0.15817346634696827
False Positives: 680 | False Negatives: 868

Run

In [ ]:
# Running KDE...
# KDE:
# ROC-AUC: 0.5524182222222221 | AUC-PR: 0.1944889957384754
# False Positives: 900 | False Negatives: 826

# Running PCA...
# PCA:
# ROC-AUC: 0.518875611111111 | AUC-PR: 0.15527543728456505
# False Positives: 900 | False Negatives: 866

# Running LOF...
# LOF:
# ROC-AUC: 0.6199740555555555 | AUC-PR: 0.22148765563972567
# False Positives: 734 | False Negatives: 798

# Running CBLOF...
# CBLOF:
# ROC-AUC: 0.5491408888888889 | AUC-PR: 0.15972168562991268
# False Positives: 900 | False Negatives: 816

# Running HBOS...
# HBOS:
# ROC-AUC: 0.5483418888888889 | AUC-PR: 0.12633453105207998
# False Positives: 504 | False Negatives: 867

# Running kNN...
# kNN:
# ROC-AUC: 0.6459407777777777 | AUC-PR: 0.23531530481594098
# False Positives: 552 | False Negatives: 818

# Running IForest...
# IForest:
# ROC-AUC: 0.5528506666666667 | AUC-PR: 0.1800694026637641
# False Positives: 899 | False Negatives: 826

# Running LODA...
# LODA:
# ROC-AUC: 0.5328418333333333 | AUC-PR: 0.15817346634696827
# False Positives: 680 | False Negatives: 868

# Running LUNAR...
# LUNAR:
# ROC-AUC: 0.7277597777777777 | AUC-PR: 0.3031987040104667
# False Positives: 900 | False Negatives: 659

In [ ]:
!python3 /content/drive/MyDrive/FTP/code/memstreamWith_FP_FN_syn.py --dataset SYN --beta 1 --memlen 16

SYN 1.0 16 0.01 5000
MemStream
ROC-AUC: 0.955265
AUC-PR: 0.8211282421895629

False Positives (l0): 701
False Negatives (l1): 72


In [ ]:
# SYN 1.0 16 0.01 5000
# MemStream
# ROC-AUC: 0.955265
# AUC-PR: 0.8211282421895629

# False Positives (l0): 701
# False Negatives (l1): 72

In [ ]:
!python3 /content/drive/MyDrive/FTP/code/memstreamWith_FP_FN.py --dataset ionosphere --beta 0.001 --memlen 4
!python3 /content/drive/MyDrive/FTP/code/memstreamWith_FP_FN.py --dataset cardio --beta 1 --memlen 64
!python3 /content/drive/MyDrive/FTP/code/memstreamWith_FP_FN.py --dataset statlog --beta 0.01 --memlen 32
!python3 /content/drive/MyDrive/FTP/code/memstreamWith_FP_FN.py --dataset satimage-2 --beta 10 --memlen 256
!python3 /content/drive/MyDrive/FTP/code/memstreamWith_FP_FN.py --dataset mammography --beta 0.1 --memlen 128
!python3 /content/drive/MyDrive/FTP/code/memstreamWith_FP_FN.py --dataset pima --beta 0.001 --memlen 64

ionosphere 0.001 4 0.01 5000
MemStream
ROC-AUC 0.8187301587301588
AUC-PR 0.6544714431018158

False Positives (l0): 163
False Negatives (l1): 20
cardio 1.0 64 0.01 5000
MemStream
ROC-AUC 0.8720784125240317
AUC-PR 0.48125306721377503

False Positives (l0): 223
False Negatives (l1): 143
statlog 0.01 32 0.01 5000
MemStream
ROC-AUC 0.7238708699199808
AUC-PR 0.6817996741894431

False Positives (l0): 1623
False Negatives (l1): 533
satimage-2 10.0 256 0.01 5000
MemStream
ROC-AUC 0.9948448541914432
AUC-PR 0.9221113289016989

False Positives (l0): 1310
False Negatives (l1): 3
mammography 0.1 128 0.01 5000
MemStream
ROC-AUC 0.902034521369869
AUC-PR 0.225807352249365

False Positives (l0): 213
False Negatives (l1): 204
pima 0.001 64 0.01 5000
MemStream
ROC-AUC 0.7407313432835821
AUC-PR 0.5530579882325852

False Positives (l0): 0
False Negatives (l1): 262


In [ ]:
# ionosphere 0.001 4 0.01 5000
# MemStream
# ROC-AUC 0.8187301587301588
# AUC-PR 0.6544714431018158

# False Positives (l0): 163
# False Negatives (l1): 20
# cardio 1.0 64 0.01 5000
# MemStream
# ROC-AUC 0.8720784125240317
# AUC-PR 0.48125306721377503

# False Positives (l0): 223
# False Negatives (l1): 143
# statlog 0.01 32 0.01 5000
# MemStream
# ROC-AUC 0.7238708699199808
# AUC-PR 0.6817996741894431

# False Positives (l0): 1623
# False Negatives (l1): 533
# satimage-2 10.0 256 0.01 5000
# MemStream
# ROC-AUC 0.9948448541914432
# AUC-PR 0.9221113289016989

# False Positives (l0): 1310
# False Negatives (l1): 3
# mammography 0.1 128 0.01 5000
# MemStream
# ROC-AUC 0.902034521369869
# AUC-PR 0.225807352249365

# False Positives (l0): 213
# False Negatives (l1): 204
# pima 0.001 64 0.01 5000
# MemStream
# ROC-AUC 0.7407313432835821
# AUC-PR 0.5530579882325852

# False Positives (l0): 0
# False Negatives (l1): 262